### Protocol Parameters

In [80]:
# The minimum collateral ratio requirement to be eligible for a loan. This is a safety measure to ensure
# that loans are overcollateralized, reducing the risk of loss for the lender. A ratio of 1.5 means that
# the collateral must be worth at least 150% of the loan value.
minimum_collateral_ratio = 1.5

# The maximum loan-to-value ratio, which is the maximum percentage of the collateral value that can be
# borrowed. A ratio of 0.5 means that the maximum loan amount is 50% of the collateral value.
maximum_loan_to_value = 1 / minimum_collateral_ratio

# Fixed fee charged for each loan transaction, representing the cost of using the protocol. This fee is
# in USD and is added to the borrowed amount when calculating the need for collateral.
protocol_usage_fee = 5  # USD

# The total amount of funds available for lending in the liquidity pool, denominated in USD.
# This pool decreases as loans are issued and increases as loans are repaid.
liquidity_pool = 30_000  # USD

# The total value of collateral held within the protocol, denominated in ADA. This is the sum of all
# collateral from all open loans.
total_collateral = 100_000  # ADA

# The threshold for liquidation expressed as a ratio; if the collateral ratio falls below this threshold
# (90% in this case), the loan becomes eligible for liquidation. This is a safety mechanism to protect
# the liquidity pool from losses due to undercollateralized loans.
liquidity_threshold = 0.9  # 90%

### Calculations

In [78]:
# Function to calculate the collateral ratio for a given amount
def calculate_collateral_ratio(collateral_value, borrowed_value, other_fees):
    # The collateral ratio is calculated by dividing the total value of the collateral
    # by the sum of the borrowed value and any other associated fees.
    # This ratio indicates the level of overcollateralization of the loan.
    return collateral_value / (borrowed_value + other_fees)

# Function to calculate the minimum amount of collateral required
def calculate_minimum_collateral_amount(borrowed_value, other_fees, minimum_collateral_ratio, ada_price):
    # First, calculate the total amount required to be covered by collateral,
    # including the borrowed value and other fees, and then apply the minimum collateral ratio.
    # This gives the minimum USD value that needs to be covered by collateral.
    minimum_amount = (borrowed_value + other_fees) * minimum_collateral_ratio
    
    # To find the minimum amount of ADA required, divide the minimum USD value
    # by the current ADA price. This conversion is necessary because the collateral
    # is provided in ADA, but the loan and fees might be denominated in another currency (e.g., USDT).
    return minimum_amount * (1 / ada_price)

def calculate_loan_to_value(loan_amount, collateral_amount, ada_price):
    # The loan-to-value ratio is calculated by dividing the loan amount by the collateral amount.
    # This ratio indicates the level of overcollateralization of the loan.
    return loan_amount / (collateral_amount * ada_price)

# Function to determine if a loan should be liquidated
def is_liquidated(collateral_value, borrowed_value, other_fees, minimum_collateral_ratio):
    # This function checks if the current collateral ratio falls below the minimum required ratio.
    # If the current collateral ratio is less than the minimum collateral ratio, the loan is undercollateralized,
    # indicating that the loan is at risk and potentially subject to liquidation.
    return calculate_collateral_ratio(collateral_value, borrowed_value, other_fees) < minimum_collateral_ratio

def calculate_loan_to_value(loan_amount, collateral_amount, ada_to_usd):
    return (loan_amount / (collateral_amount * ada_to_usd)) * 100

def can_be_liquidated_due_to_high_ltv(loan_to_value, maximum_loan_to_value):
    return loan_to_value > maximum_loan_to_value

### Sample Collateral Ratio

In [81]:
ada_collateral = 1515 
ada_to_usd = 0.3 
loan_amount = 500 # USD
collateral_ratio = calculate_collateral_ratio(ada_collateral * ada_to_usd, loan_amount, protocol_usage_fee)
minimum_collateral_amount = calculate_minimum_collateral_amount(loan_amount, protocol_usage_fee, minimum_collateral_ratio, ada_to_usd)
loan_to_value = calculate_loan_to_value(loan_amount, ada_collateral, ada_to_usd)

print("Collateral Ratio: ", collateral_ratio)
print("Ada to usdt exchange rate: ", ada_to_usd)
print("Minimum Collateral Amount: ", minimum_collateral_amount)
print("Maximum Laon to Value: ", maximum_loan_to_value)
print("Loan to Value: ", loan_to_value)
print("Can be liquidated due to high ltv: ", can_be_liquidated_due_to_high_ltv(loan_to_value, maximum_loan_to_value))
print("Is Liquidated: ", is_liquidated(ada_collateral * ada_to_usd, loan_amount, protocol_usage_fee, minimum_collateral_ratio))

Collateral Ratio:  0.9
Ada to usdt exchange rate:  0.3
Minimum Collateral Amount:  2525.0
Loan to Value:  110.01100110011002
Can be liquidated due to high ltv:  True
Is Liquidated:  True
